In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset=pd.read_csv('PP_Mtrls_and_their_Mchncl_Prprts.csv')
dataset.head()

,Material,Su,Sy,E,G,mu,Ro,Use
0,ANSI Steel SAE 1015 asrolled,421,314,207000,79000,0.3,7860,True
1,ANSI Steel SAE 1015 normalized,424,324,207000,79000,0.3,7860,True
2,ANSI Steel SAE 1015 annealed,386,284,207000,79000,0.3,7860,True
3,ANSI Steel SAE 1020 asrolled,448,331,207000,79000,0.3,7860,True
4,ANSI Steel SAE 1020 normalized,441,346,207000,79000,0.3,7860,True


In [3]:
dataset.drop('Material', axis=1, inplace=True)

In [4]:
columns_to_drop = ['Sy','mu']
dataset = dataset.drop(columns=columns_to_drop)
dataset.head()

,Su,E,G,Ro,Use
0,421,207000,79000,7860,True
1,424,207000,79000,7860,True
2,386,207000,79000,7860,True
3,448,207000,79000,7860,True
4,441,207000,79000,7860,True


In [5]:
dataset['Use'] = dataset['Use'].astype('object')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Su      802 non-null    int64 
 1   E       802 non-null    int64 
 2   G       802 non-null    int64 
 3   Ro      802 non-null    int64 
 4   Use     802 non-null    object
dtypes: int64(4), object(1)
memory usage: 31.5+ KB


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset.head()

,Su,E,G,Ro,Use_True
0,421,207000,79000,7860,True
1,424,207000,79000,7860,True
2,386,207000,79000,7860,True
3,448,207000,79000,7860,True
4,441,207000,79000,7860,True


In [7]:
label_encoder = LabelEncoder()

# Fit and transform the target column
dataset['Use_True'] = label_encoder.fit_transform(dataset['Use_True'])
dataset

,Su,E,G,Ro,Use_True
0,421,207000,79000,7860,1
1,424,207000,79000,7860,1
2,386,207000,79000,7860,1
3,448,207000,79000,7860,1
4,441,207000,79000,7860,1
...,...,...,...,...,...
797,980,206000,80000,7860,0
798,1100,206000,80000,7860,0
799,1200,206000,80000,7860,0
800,1030,206000,80000,7860,0


In [8]:
indep=dataset[['Su','E','G','Ro']]
dep=dataset["Use_True"]

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.svm import SVC

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['linear','rbf','poly','sigmoid'],
             'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]} 
grid = GridSearchCV(SVC(probability=False), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[249   6]
 [  5   8]]


In [14]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       255
           1       0.57      0.62      0.59        13

    accuracy                           0.96       268
   macro avg       0.78      0.80      0.79       268
weighted avg       0.96      0.96      0.96       268



In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 1000, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9596749932937365


In [16]:
from sklearn.metrics import roc_auc_score
y_scores = grid.decision_function(X_test)
roc_auc_score=roc_auc_score(y_test,y_scores)
print("The roc_auc_score is:",roc_auc_score)

The roc_auc_score is: 0.9826546003016592


In [17]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001829,0.001461,0.002172,0.002673,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.930466,0.925773,0.916697,0.940628,0.929815,0.928676,0.007737,31
1,0.001294,0.000762,0.001544,0.001444,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.930466,0.930466,0.916697,0.924155,0.929815,0.926319,0.005364,34
2,0.001549,0.001197,0.001434,0.000930,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.930466,0.930466,0.916697,0.916697,0.929815,0.924828,0.006643,37
3,0.002080,0.001073,0.001768,0.001072,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.925773,0.930466,0.916697,0.916697,0.929815,0.923889,0.006089,39
4,0.002357,0.001366,0.001312,0.000787,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.930466,0.925773,0.916697,0.940628,0.929815,0.928676,0.007737,31
5,0.001980,0.000564,0.001761,0.000682,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.930466,0.930466,0.916697,0.924155,0.929815,0.926319,0.005364,34
6,0.002172,0.000947,0.001477,0.000705,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.930466,0.930466,0.916697,0.916697,0.929815,0.924828,0.006643,37
7,0.001765,0.000685,0.001938,0.001551,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.925773,0.930466,0.916697,0.916697,0.929815,0.923889,0.006089,39
8,0.001001,0.000293,0.000555,0.000046,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.930466,0.937188,0.916697,0.943370,0.936598,0.932864,0.009057,29
9,0.001321,0.000804,0.001162,0.000773,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.950515,0.962617,0.916697,0.952413,0.962264,0.948901,0.016845,11


In [18]:
su_input=float(input("Ultimate Tensile Strength (Su) in MPa:"))
E_input=float(input("Elastic Modulus (E) in MPa:"))
g_input=float(input("Shear Modulus (G) in MPa:"))
ro_input=float(input("Density (Ro) in Kg/m3:"))

Ultimate Tensile Strength (Su) in MPa:1000
Elastic Modulus (E) in MPa:119876
Shear Modulus (G) in MPa:67543
Density (Ro) in Kg/m3:8912


In [19]:
Future_Prediction=grid.predict([[su_input,E_input,g_input,ro_input]])
y=("Future_Prediction={}".format(Future_Prediction))
if (y==0):
    print("Future_Prediction=[False]")
else:
    print('Future_Prediction=[True]')

Future_Prediction=[True]
